In [27]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

In [28]:
# #Move indicator into its own Class so that will show up in output file. 
# #Reference: https://www.backtrader.com/docu/inddev.html
# class RSI_Oversold(bt.Indicator):
#     lines = ('oversold',)
    
#     params = (
#         ('level', 30),    
#     )

#     def __init__(self):
#         self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

In [29]:
# class RSI_Overbought(bt.Indicator):
#     lines = ('overbought',)
    
#     params = (
#         ('level', 70),    
#     )

#     def __init__(self):
#         self.lines.overbought = bt.indicators.RSI(self.data) > self.p.level

In [34]:
# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7), # set to 7 as default, can modify this in the future
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        
        # Add RSI
        self.rsi = bt.indicators.RSI(self.stock_px)
#         self.rsi_oversold = RSI_Oversold(self.stock_px)
#         self.rsi_overbought = RSI_Overbought(self.stock_px)
        self.rsi_oversold = self.rsi < 30
        self.rsi_overbought = self.rsi > 70
        
        # Add LaguerreRSI
        self.lrsi = bt.indicators.LRSI(self.stock_px)
        
        # Add TSI
        self.tsi = bt.indicators.TSI(self.stock_px)
        
#         # Add WilliamsAD & WilliamsR
#         self.will_AD = bt.indicators.WilliamsAD(self.stock_px)
#         self.will_R = bt.indicators.WilliamsR(self.stock_px)
        
        # Add UltimateOscillator
        self.ulti_osc = bt.indicators.UltimateOscillator(self.stock_px)
        
        # Add

        self.in_trade = False
        self.days_in_trade = 0
        self.flag = 0   # -1 sell, 0 no pos, 1 buy.
        
    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size

#         # If not in the trade
#         if (self.flag == 0):
#             if (self.rsi_oversold[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 print("Oversold indicator triggered...entering position")
#                 self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = 1
#             elif (self.rsi_overbought[0] > 0 and self.days_in_trade < self.p.max_days_in_trade):
#                 print("Overbought indicator triggered...entering position")
#                 self.sell(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#                 self.flag = -1
        
#         # If in the long position
#         elif (self.flag == 1):
#             # Close the position if the SELL signal triggered or reached the max days to hold
#             if (self.rsi_overbought[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.rsi_oversold[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
#                 print("In position %d days" % self.days_in_trade)
        
#         # If in the short position
#         elif (self.flag == -1):
#             # Close the position if the BUY signal triggered or reached the max days to hold
#             if (self.rsi_oversold[0] > 0 or self.days_in_trade == self.p.max_days_in_trade):
#                 self.close(exectype=bt.Order.Close)
#                 print("Closing the position")
#                 self.days_in_trade = 0
#                 self.flag = 0
                    
#             elif (self.rsi_overbought[0] > 0 or self.days_in_trade < self.p.max_days_in_trade):
#                 self.days_in_trade += 1
#                 print("In position %d days" % self.days_in_trade)
            
        
        # Check if we are currently in a trade
        if(not size):
            #If not has our indicator triggered?
            if (self.rsi_oversold[0] > 0):
#             if (self.rsi_oversold[0] > 0 or self.rsi_overbought[0] > 0):
#                 print("Oversold/bought indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
                
            if self.rsi_overbought[0] > 0:
#                 print("Overbought indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                               
        else:
            self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            
            if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0

In [35]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../../datas/MSFT.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' # hist/implied vol
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Set the commission - 0.1% ... divide by 100 to remove the %
#     cerebro.broker.setcommission(commission=0.001)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='MSFT-TI-Vanessa.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


AttributeError: 'Lines_LineSeries_LineSeriesStub' object has no attribute 'close'